In [1]:
import torch
import torch.nn as nn
import numpy as np
import pickle
import torch
import scanpy as sc
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys

from model.stDiff_model import DiT_stDiff
from utils import save_log_file, save_model, SinkhornLoss

/home/hanyuji/miniconda3/envs/DYffusion/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gene_num = 50
depth = 6
hidden_size = 512
head = 16
device = torch.device('cuda:1')

model_interpolation = DiT_stDiff(
    input_size=gene_num * 2,
    output_size=gene_num,
    hidden_size=hidden_size,
    depth=depth,
    num_heads=head,
    classes=6,
    dit_type='dit',
    mlp_ratio=4.0,
).to(device)

model_forcast = DiT_stDiff(
    input_size=gene_num,
    output_size=gene_num,
    hidden_size=hidden_size,
    depth=depth,
    num_heads=head,
    classes=6,
    dit_type='dit',
    mlp_ratio=4.0,
).to(device)



In [10]:
# schedule
time_step_n = 10
time_step = [i / time_step_n for i in range(time_step_n + 1)]
time_step = time_step[1:]

DYff_step_n = 10
DYff_step = [i / time_step_n for i in range(time_step_n + 1)]
DYff_step = DYff_step
DYff_schedule = [torch.full((32,), item).to(device) for item in DYff_step]

# 改成距离最后一个点的距离？
init_step = torch.rand(32,).to(device)

# 模拟数据
init_data = torch.rand(32,50).to(device)

In [11]:
# 初始化last

pred_last = model_forcast(x=init_data, t=init_step)


In [12]:
for cur_step in DYff_schedule:
    pred_mid = model_interpolation(x0=init_data, x2=pred_last, t=cur_step, mode='interpolation')
    pred_last = model_forcast(pred_mid)


AttributeError: 'NoneType' object has no attribute 'device'

In [ ]:
# refinement
for cur_step in time_step:
    pred_mid = model_interpolation(init_data,pred_last)


In [ ]:
init_data.shape

torch.Size([32, 50])

In [ ]:
torch.cat((pred_last, init_data), dim=-1).shape

torch.Size([32, 100])

In [6]:
cur_step

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:1')

In [ ]:

model_interpolation(x0=init_data, x2=pred_last, t=cur_step, mode='interpolation')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x50 and 100x512)